In [17]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    auc,
    RocCurveDisplay,
    roc_curve,
)
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from config import CONFIG
from utils import create_model

In [14]:
DATA_DIR = "dataset/1-raw-v2"
class_names = os.listdir(DATA_DIR)

train_set, test_set = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    seed=CONFIG["seed_value"],
    image_size=CONFIG["img_shape"],
    batch_size=CONFIG["batch_size"],
    label_mode="categorical",
    validation_split=0.2,
    subset="both",
)

train_images, train_labels = [], []
for images, labels in train_set:
    train_images.append(images)
    train_labels.append(labels)

train_images = np.concatenate(train_images).astype("uint8")
train_labels = np.concatenate(train_labels)

train_pairs = zip(train_images, train_labels)

test_images, test_labels = [], []
for images, labels in test_set:
    test_images.append(images)
    test_labels.append(labels)

test_images = np.concatenate(test_images).astype("uint8")
test_labels = np.concatenate(test_labels)

Found 402 files belonging to 2 classes.
Using 322 files for training.
Using 80 files for validation.


In [15]:
mobilenetv3small_checkpoints_path = "checkpoints/mobilenetv3small"

mobilenetv3small_latest_checkpoint = tf.train.latest_checkpoint(
    mobilenetv3small_checkpoints_path
)

# Create base model
mobilenetv3small_model = tf.keras.applications.MobileNetV3Small(
    input_shape=CONFIG["input_shape"],
    include_top=False,
    classes=CONFIG["num_classes"],
    pooling="avg",
)

# Create custom models
model = create_model(mobilenetv3small_model, CONFIG)

# Load model weights
model.load_weights(mobilenetv3small_latest_checkpoint).expect_partial()

In [86]:
# Define the K-fold Cross Validator with 5 folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=CONFIG["seed_value"])

train_tprs = []
train_aucs = []
train_mean_fpr = np.linspace(0, 1, 100)

valid_tprs = []
valid_aucs = []
valid_mean_fpr = np.linspace(0, 1, 100)

# Loop over the dataset to create separate folds
for i, (train_idx, valid_idx) in enumerate(
    cv.split(train_images, np.argmax(train_labels, axis=1))
):
    X_train, y_train = train_images[train_idx], train_labels[train_idx]
    X_valid, y_valid = train_images[valid_idx], train_labels[valid_idx]

    y_train_preds = model.predict(X_train)[:, 1].tolist()
    y_valid_preds = model.predict(X_valid)[:, 1].tolist()

    fpr, tpr, thresholds = roc_curve(y_train.argmax(axis=1), y_train_preds)
    train_tprs.append(np.interp(train_mean_fpr, fpr, tpr))
    train_aucs.append(roc_auc)

    fpr, tpr, thresholds = roc_curve(y_valid.argmax(axis=1), y_valid_preds)
    valid_tprs.append(np.interp(valid_mean_fpr, fpr, tpr))
    valid_aucs.append(roc_auc)

test_preds = model.predict(test_images)[:, 1].tolist()
test_fpr, test_tpr, test_thresholds = roc_curve(test_labels.argmax(axis=1), test_preds)

3/3 [==============================] - 0s 95ms/step


In [102]:
fig, ax = plt.subplots(figsize=(20, 10), dpi=600)
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 18
plt.rcParams["axes.titlepad"] = 15

train_mean_tpr = np.mean(train_tprs, axis=0)
train_mean_tpr[0] = 0.0
valid_mean_tpr = np.mean(valid_tprs, axis=0)
valid_mean_tpr[0] = 0.0

train_roc_auc = auc(train_mean_fpr, train_mean_tpr)

plt.plot(
    train_mean_fpr,
    train_mean_tpr,
    lw=2,
    label="Training (AUC = %0.2f)" % (train_roc_auc),
)

valid_roc_auc = auc(valid_mean_fpr, valid_mean_tpr)
plt.plot(
    valid_mean_fpr,
    valid_mean_tpr,
    lw=2,
    label="Validation (AUC = %0.2f)" % (valid_roc_auc),
)

test_roc_auc = auc(test_fpr, test_tpr)
plt.plot(
    test_fpr,
    test_tpr,
    lw=2,
    label="Test (AUC = %0.2f)" % (test_roc_auc),
)

plt.plot([0, 1], [0, 1], linestyle="--", lw=2, color="black")
plt.xlabel("False Positive Rate", fontsize=22)
plt.ylabel("True Positive Rate", fontsize=22)
plt.title("ROC Curve", fontsize=22)
plt.legend(loc="lower right")
plt.xlim([-0.005, 1])
plt.ylim([0, 1.01])
plt.grid()

plt.show()

: 